## Photo Tagger in SageMaker Training

We will now upload the dataset to Amazon S3...

In [2]:
import sagemaker, boto3
from sagemaker.session import Session
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()
prefix = 'my-house-rooms/train'
print(bucket)

sagemaker-eu-west-1-889960878219


In [3]:
inputs = sess.upload_data(path='Split_Dataset/train', bucket=bucket, key_prefix=prefix)
print("input spec (in this case, just an S3 path): {}".format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-eu-west-1-889960878219/my-house-rooms/train


In [13]:
!pygmentize code/train.py

import argparse
import json
import logging
import os
import sys

#import sagemaker_containers
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
from torchvision import datasets, transforms

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))


# Based on https://github.com/pytorch/examples/blob/master/mnist/main.py
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 32

In [9]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point='code/train.py',
    role=role,
    py_version="py36",
    framework_version="1.8",
    instance_count=2,
    instance_type="ml.g4dn.2xlarge",
    hyperparameters={
        "epochs": 5,
        "batch-size": 32,
        "backend": "gloo"
    },
)

In [ ]:
estimator.fit({"training": inputs})

In [ ]:
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large"
)